In [1]:
#Importing all the key libraries
import os
os.environ['WANDB_API_KEY'] = 'abc3e61cc8dfd07d2ba9a8661065c6de5303174d' #change it to your wandb API
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
from PIL import Image
from collections import Counter

import numpy as np
import matplotlib.pyplot as plt
import random
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models
import wandb
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.data import random_split
from torch.utils.data import SubsetRandomSampler
from torchvision.transforms import ToTensor

## Utility code

In [2]:
class CustomImageDataset(torch.utils.data.Dataset):
    """ Custom Image dataset which overrides torch dataset class. Initialises 
    the dataset and enables us to code the logic of retrieving images and labels from
    the directory structure we have
    """
    def __init__(self, dir, limit=None, indices=None, transform=None):
        self.data_dir = dir
        self.limit = int(1e10) if limit is None else limit
        self.images = []
        self.labels = []
        self.label_names = []
        i = 0
        for c in os.listdir(dir):
          ims = os.listdir(os.path.join(dir,c))[:self.limit]
          ims = [os.path.join(dir,c,im) for im in ims]
          self.images.extend(ims)
          self.labels.extend([i]*len(ims))
          self.label_names.append(c)
          i += 1
            
        if indices is not None:
            self.images = [self.images[i] for i in indices]
            self.labels = [self.labels[i] for i in indices]
        self.transform = transform

    # Defining the length of the dataset
    def __len__(self):
        return len(self.images)

    # Defining the method to get an item from the dataset
    def __getitem__(self, index):
        image = Image.open(self.images[index])
        image = image.convert("RGB")
        label = self.labels[index]
        # Applying the transform
        if self.transform:
            image = self.transform(image)

        return image, label

In [3]:
def split_validation(label_array, valid_size = 0.1, seed = 42):
  """The functions splits the data into train and validation stratifiedly
  """
  train_idx, test_idx = [],[]
  for i,ci in enumerate(np.unique(label_array)):
      indices = np.where(label_array==ci)[0]
      train_len = int(indices.shape[0]*(1-valid_size))
      shuffled = np.random.RandomState(seed+i).permutation(indices)
      train_idx.extend(shuffled[:train_len])
      test_idx.extend(shuffled[train_len:])
  return train_idx,test_idx

In [4]:
def set_seed(seed=42):
    """Sets random seeds for reproducibility.
    """
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False  
    
    torch.cuda.manual_seed_all(seed)

In [5]:
class Trainer:
  """ Trainer agent which handles the training
  """
  def __init__(self):
    """ Initializing agent parameters
    """
    self.input_size = (224,224) 
    self.criterion = nn.CrossEntropyLoss()
    self.optimizer = None
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.model = None
    self.train_loader = None
    self.val_loader = None
    self.test_ds = None
    self.test_loader = None

  def create_model(self):
    """ Fetches a pretrained model and takes care of freezing and unfreezing layers"
    """
    num_classes = 10
    model = models.resnet50(weights='DEFAULT')
    for param in model.parameters():
        param.requires_grad = False  # freeze everything
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, num_classes)
        )
    for param in model.layer4.parameters():
        param.requires_grad = True
    for param in model.fc.parameters():
        param.requires_grad = True  # train only the head

    model.to(self.device)
    self.optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    self.model = model

  def create_data(self, batch_size, data_aug=False, limit=None, train=False):
      """ Creates train, val, test datasets with augmentation transforms if required. 
      The limit controls how much of the dataset that we want to choose
      """
      TRAIN_DATA_PATH = "/kaggle/input/inaturalist/inaturalist_12K/train"
      TEST_DATA_PATH = "/kaggle/input/inaturalist/inaturalist_12K/val"

      base_transform = transforms.Compose([
                transforms.Resize(self.input_size),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
              ])
      if data_aug:
          aug_transform = transforms.Compose([
                transforms.Resize(self.input_size),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
                transforms.RandomRotation(degrees=10),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
              ])
      else:
          aug_transform = base_transform

      # we apply data augmentation only to the train dataset
      if train:
        full_dataset = CustomImageDataset(
            TRAIN_DATA_PATH, limit=limit, transform=None)
        train_idx, val_idx = split_validation(
            full_dataset.labels, valid_size=0.2, seed=42)
        train_dataset = CustomImageDataset(
            TRAIN_DATA_PATH, limit=limit, transform=aug_transform, indices=train_idx)
        val_dataset = CustomImageDataset(
            TRAIN_DATA_PATH, limit=limit, transform=base_transform, indices=val_idx)
        self.train_loader = DataLoader(
            dataset=train_dataset, batch_size=batch_size,
            shuffle=True, num_workers=2)
        self.val_loader = DataLoader(
            dataset=val_dataset, batch_size=batch_size,
            num_workers=2)
      else:
        self.test_ds = CustomImageDataset(
            TEST_DATA_PATH, limit=limit, transform=base_transform)
        self.test_loader = DataLoader(
          num_workers=2,dataset=self.test_ds, batch_size=batch_size)

  def worker_init_fn(self, worker_id):
      """ Set worker seed for reproducibility 
      """
      np.random.seed(torch.initial_seed() % 2**32)

  def run_step(self, dataloader, is_train=False):
      """ Runs one step of optimization if needed and evaluates the model
      """
      correct = 0
      total = 0
      cum_loss = 0
      if is_train:
        self.model.train()
      else:
        self.model.eval()

      for batch_index, (data, targets) in enumerate(tqdm(dataloader)):
          data = data.to(self.device)
          targets = targets.to(self.device)

          with torch.set_grad_enabled(is_train):
            outputs = self.model(data)
            loss = self.criterion(outputs, targets)
            cum_loss += loss.item()
            _, preds = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (preds == targets).sum().item()
            if is_train:
              self.optimizer.zero_grad()
              loss.backward()
              self.optimizer.step()
      acc = correct/total
      return cum_loss, acc

  def run_train_pipeline(self, n_epochs, limit=None, use_wandb=False, config=None):
      """ Pipeline setup for the whole training process, includes data creation, model
      creation, integration with wandb and model evaluation
      """
      if use_wandb:
        run = wandb.run
      else:
        run = None

      set_seed(42)
      batch_size = config.batch_size
      data_aug = config.data_aug

      self.create_model()
      self.create_data(batch_size, data_aug=data_aug, limit=limit, train=True)

      for epoch in range(n_epochs):
        train_loss, train_acc = self.run_step(self.train_loader, is_train=True)
        val_loss, val_acc = self.run_step(self.val_loader)
        if run:
          wandb.log({
              'epoch':epoch,
              'train_loss':train_loss, 'train_acc':train_acc,
              'val_loss':val_loss, 'val_acc':val_acc
          })
        else:
          print({
              'epoch':epoch,
              'train_loss':train_loss, 'train_acc':train_acc,
              'val_loss':val_loss, 'val_acc':val_acc
          })


          
  def test(self):
      batch_size = 128
      self.create_data(batch_size)
      self.model.eval()
      test_loss,test_acc = self.run_step(self.test_loader)
      print(f"Loss on test set : {test_loss}")
      print(f"Accuracy on test set : {test_acc}")


## Model training and evaluation

In [6]:
from types import SimpleNamespace
batch_size = 128
data_aug = True

config = SimpleNamespace(
   batch_size=batch_size,
   data_aug = data_aug
)

In [7]:
n_epochs = 30
tr = Trainer()
tr.run_train_pipeline(n_epochs=n_epochs, limit=None, use_wandb=False, config=config)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 220MB/s]
100%|██████████| 16/16 [00:18<00:00,  1.13s/it]


{'epoch': 0, 'train_loss': 143.8648202419281, 'train_acc': 0.17189648706088262, 'val_loss': 36.02696657180786, 'val_acc': 0.31}


100%|██████████| 16/16 [00:13<00:00,  1.21it/s]


{'epoch': 1, 'train_loss': 139.27291679382324, 'train_acc': 0.39704963120390047, 'val_loss': 34.696574211120605, 'val_acc': 0.534}


100%|██████████| 16/16 [00:13<00:00,  1.17it/s]


...

{'epoch': 27, 'train_loss': 19.81357392668724, 'train_acc': 0.897362170271284, 'val_loss': 7.786284267902374, 'val_acc': 0.8405}


100%|██████████| 16/16 [00:13<00:00,  1.17it/s]


{'epoch': 28, 'train_loss': 18.88368183374405, 'train_acc': 0.9046130766345793, 'val_loss': 7.806801348924637, 'val_acc': 0.844}


100%|██████████| 16/16 [00:13<00:00,  1.17it/s]

{'epoch': 29, 'train_loss': 18.143728151917458, 'train_acc': 0.9083635454431804, 'val_loss': 7.801267504692078, 'val_acc': 0.8455}


In [8]:
tr.test()

100%|██████████| 16/16 [00:17<00:00,  1.08s/it]

Loss on test set : 7.39499905705452
Accuracy on test set : 0.8555
